In [1]:
# https://adventofcode.com/2021/day/13
import time

import numpy as np

In [2]:
def load_input(is_sample=True):
    if is_sample:
        txt_path = "day13_sample.txt"
    else:
        txt_path = "day13_.txt"
    with open(txt_path) as f:
        dots = []
        while True:
            line = f.readline()
            # read until line is '' or EOF
            if (not line) or (len(line) == 1):
                break
            x, y = line.split(',')
            dots.append([int(x), int(y)])

        instructions = []
        while True:
            line = f.readline()
            if not line:
                break
            xy, value = line.replace('fold along ', '').split("=")
            instructions.append([xy, int(value)])
    
    return np.array(dots), instructions 

In [3]:
def fold_y_half(field):
    field_ = field.copy()
    y_max = np.shape(field)[0]
    assert y_max % 2 == 1, 'row of the field should be odd number.'
    
    # n should be smaller than the half of the field.
    n = int((y_max-1)/2)
    for i in range(n):
        field_[n-i-1, :] += field_[n+i+1, :]
    field_ = np.where(field_>0, 1, 0)
    return field_[:n, :]


def fold_y(field, n):
    n_max, m_max = np.shape(field)

    # the length of folded parts are x and x_.
    n_ = n_max - n - 1
    
    if n <= n_ :
        filler = np.zeros(((n_ - n), m_max))
        field_filled = np.vstack((filler, field))
        field_half = fold_y_half(field_filled)
        field_ = field_half[:n_, :]
    else:
        filler = np.zeros((2*(n+1) - n_max - 1, m_max))
        field_filled = np.vstack((field, filler))
        field_half = fold_y_half(field_filled)
        field_ = field_half[:n, :]
    return field_


def fold_x(field, n):
    return fold_y(field.T, n).T


def fold(field, instruction):
    if instruction[0] == 'x':
        field_ = fold_x(field, instruction[1])
    elif instruction[0] == 'y':
        field_ = fold_y(field, instruction[1])
    return field_

In [4]:
dots, instructions = load_input(is_sample=False)

# set values.
x_max = np.max(dots[:, 1])+1
y_max = np.max(dots[:, 0])+1
field = np.zeros((x_max, y_max))
for dot in dots:
    field[dot[1], dot[0]]=1

## part 1.
field = fold(field, instructions[0])
print(f"number of visible dots: {np.sum(field)}")

## part 2.
for instruction in instructions:
    field = fold(field, instruction)
    
# display the result.
for line in field:
    line = np.where(line==1, '*', ' ')
    print(''.join(line.tolist()))

number of visible dots: 850
 **  *  *  **   **  ***   **   **  *  * 
*  * *  * *  * *  * *  * *  * *  * *  * 
*  * **** *    *    *  * *    *  * *  * 
**** *  * * ** *    ***  * ** **** *  * 
*  * *  * *  * *  * *    *  * *  * *  * 
*  * *  *  ***  **  *     *** *  *  **  
